In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import Adam
from keras.applications.vgg19 import VGG19

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
weights_path = 'vgg16_weights.h5'
top_model_weights_path = 'top_model_weights_path.h5'

In [3]:
epochs = 10
batch_size = 24
split=0.9

In [4]:
X_train=np.load('X_train_rotate.npy')
target_train=np.load('target_train.npy')

In [5]:
def get_callbacks(filepath, patience):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

In [6]:
def model_plus():
    base_model = applications.VGG16(weights='imagenet', include_top=False,input_shape=(75,75,3))
    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(512, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(1, activation='sigmoid'))
    # note that it is necessary to start with a fully-trained
    # classifier, including the top classifier,
    # in order to successfully do fine-tuning
    top_model.load_weights(top_model_weights_path)
    # add the model on top of the convolutional base
    #model.add(top_model)
    model = Model(inputs=base_model.input, outputs=top_model(base_model.output))
    # set the first 25 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in model.layers[:-5]:
        layer.trainable = False
    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    mypotim=Adam(lr=0.00001, beta_1=0.5, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    return model

In [7]:
file_path = "model_weights.hdf5"

In [8]:
#5fold cross validation

from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

seed = 7
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=6, shuffle=True, random_state=seed)
cvscores = []
cvscores_ = []
for train, test in kfold.split(X_train, target_train):
    model = model_plus()
    model.fit(X_train[train], target_train[train],
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=0,
                    validation_data=(X_train[test], target_train[test]),
                    callbacks = get_callbacks(file_path,5))
    model.load_weights(filepath=file_path)
    scores = model.evaluate(X_train[test], target_train[test], verbose=0)
    print(model.metrics_names[0], scores[0])
    print(model.metrics_names[1], scores[1])
    cvscores.append(scores[0])
    cvscores_.append(scores[1])

loss 0.23306351953
acc 0.889165628892
loss 0.180106575358
acc 0.927770859352
loss 0.18496352757
acc 0.929016189364
loss 0.231377089273
acc 0.897627965044
loss 0.201458270407
acc 0.896379525593
loss 0.262616360952
acc 0.885143570537


In [9]:
print('Test loss average:',np.mean(cvscores), 'Test loss std:',np.std(cvscores))
print('Test acc average:',np.mean(cvscores_), 'Test loss std:',np.std(cvscores_))

Test loss average: 0.215597557182 Test loss std: 0.029333696134
Test acc average: 0.904183956464 Test loss std: 0.0176302694379


In [10]:
'print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

SyntaxError: EOL while scanning string literal (<ipython-input-10-24da91851f5b>, line 1)

In [ ]:
X_test=np.load('X_test.npy')
predicted_test=model.predict(X_test,verbose=1)

In [ ]:
import pandas as pd
submission = pd.DataFrame()
test_id=np.load('test_id.npy')
submission['id']=test_id
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)

In [ ]:
i=0
for layer in model.layers:
    i=i+1
    
print (i)    